### Retrieval-Augmented Generation with Wikipedia

In [1]:
import os
import openai
from gpt_helper import set_openai_api_key_from_txt,GPTchatClass,printmd
from wiki_helper import wiki_search
from util import printmd,extract_quoted_words
print ("openai version:[%s]"%(openai.__version__))

openai version:[0.28.0]


### Instantiate GPT Agent

In [2]:
set_openai_api_key_from_txt(key_path='../key/rilab_key.txt')
GPT = GPTchatClass(
    gpt_model='gpt-3.5-turbo', # 'gpt-3.5-turbo' / 'gpt-4'
    role_msg='Your are a helpful assistant summarizing infromation and answering user queries.')

OpenAI API Key Ready from [../key/rilab_key.txt].
Chat agent using [gpt-3.5-turbo] initialized with the follow role:[Your are a helpful assistant summarizing infromation and answering user queries.]


### Our RAG agent will use the following strategies
We assume that a user question is given (e.g., 'Who is the current president of South Korea?').
* Step 1. For the given question, our `GPT agent` will first generate a number of entities for searching Wikipedia.
* Step 2. Then, our `WikiBot` will provide (i.e., crawl) related information summarized with the `GPT agent` considering the user question.
* Step 3. Finally, the summarized texts and the original user question will be given to the `GPT agent` to answer. 

In [3]:
question = 'Who is the current president of South Korea?'
"""
question = '''
    I am an interactive humanoid robot agent. 
    I have following action capabilites:['idle','waving','greeting','raising hands','hugging','reading a book']
    I can detect following observations:['no people','a person appears','a person waves hands','a person leaves']
    I have a following personality:['Introverted and Childish']
    What is the best next action when I am in ['idle'] state and observes ['a person waves hands']?
'''
"""
print ("question: %s"%(question))

question: Who is the current president of South Korea?


### Step 1. Generate entities for wiki search

In [4]:
user_msg = \
    """
    Suppose you will use Wikipedia for retrieving information. 
    Could you recommend three query words wrapped with quotation marks considering the following question?
    """ + '"' + question + '"'

In [5]:
response_content = GPT.chat(
    user_msg=user_msg,PRINT_USER_MSG=True,PRINT_GPT_OUTPUT=True,
    RESET_CHAT=True,RETURN_RESPONSE=True)

[USER_MSG]



    Suppose you will use Wikipedia for retrieving information. 
    Could you recommend three query words wrapped with quotation marks considering the following question?
    "Who is the current president of South Korea?"

[GPT_OUTPUT]


Sure! Here are three query words you can use to search for the current president of South Korea on Wikipedia:

1. "Current president of South Korea"
2. "President of South Korea"
3. "South Korean president"

By putting these phrases in quotation marks, it will help narrow down the search results and prioritize pages that contain the exact phrase you're looking for.

In [6]:
# Print summarized sentence with a markdown format
printmd(response_content)

Sure! Here are three query words you can use to search for the current president of South Korea on Wikipedia:

1. "Current president of South Korea"
2. "President of South Korea"
3. "South Korean president"

By putting these phrases in quotation marks, it will help narrow down the search results and prioritize pages that contain the exact phrase you're looking for.

In [7]:
entities = extract_quoted_words(response_content)
if len(entities) > 3: entities = entities[-3:]
print (entities)

['Current president of South Korea', 'President of South Korea', 'South Korean president']


### Step 2. Query entities to `WikiBot`

In [8]:
paragraphs_return = []
for entity in entities:
    paragraphs_return += wiki_search(entity=entity,VERBOSE=True)

entity:[Current president of South Korea] mismatched. use [President of South Korea] instead.
 We have total [293] paragraphs.
 After filtering, we have [31] and [8] paragraphs returned (k:[5] and m:[3])
entity:[President of South Korea] matched.
 We have total [293] paragraphs.
 After filtering, we have [31] and [8] paragraphs returned (k:[5] and m:[3])
entity:[South Korean president] matched.
 We have total [293] paragraphs.
 After filtering, we have [31] and [8] paragraphs returned (k:[5] and m:[3])


In [9]:
# Get the unique elements
paragraphs_unique = list(set(paragraphs_return))
print ("Number of paragraphs [%d] => unique ones [%d]"%
       (len(paragraphs_return),len(paragraphs_unique)))

Number of paragraphs [24] => unique ones [8]


In [10]:
# Now summarize each paragraph into a single sentence considering the question
summarized_sentences = []
for p_idx,p in enumerate(paragraphs_unique):
    user_msg = "You are given following question: "+question
    user_msg += "Could you summarize the following paragraph into one setence? \n "+p
    response_content = GPT.chat(
        user_msg=user_msg,PRINT_USER_MSG=False,PRINT_GPT_OUTPUT=False,
        RESET_CHAT=True,RETURN_RESPONSE=True)
    # Append summarized sentences
    summarized_sentences.append(response_content)
    # Print summarized sentence with a markdown format
    printmd(response_content)

The current president of South Korea is directly elected for a five-year term with no possibility of re-election, and in case of a vacancy, a successor must be elected within sixty days.

The current president of South Korea is the head of state and government, leading the State Council and serving as the commander-in-chief of the Armed Forces.

The current president of South Korea is not mentioned in the given paragraph

The presidential term in South Korea is currently set at five years since 1988, with the president being barred from re-election since 1981.

The Provisional Government of the Republic of Korea established in September 1919 was recognized and succeeded by South Korea and its current Constitution.

The paragraph describes the National Security Council and the Peaceful Unification Advisory Council in South Korea, their roles and membership.

Yoon Suk Yeol, a former prosecutor general and member of the conservative People Power Party, became the president of South Korea on May 10, 2022, after winning the 2022 presidential election with a narrow 48.5% of the votes, defeating Lee Jae-myung from the Democratic Party.

The paragraph discusses the controversial Advisory Council of Elder Statesmen in South Korea, which was expanded and elevated to cabinet rank before Roh Tae Woo became president, leading to suspicions of it being designed to benefit a specific individual. However, these suspicions became irrelevant when former President Chun withdrew from politics in November 1988.

### Step 3. Answer the question using `summarized_sentences`

In [11]:
user_msg = " ".join(summarized_sentences)
user_msg += " Using the information above, could you answer the following question? "
user_msg += question

In [12]:
response_content = GPT.chat(
    user_msg=user_msg,PRINT_USER_MSG=True,PRINT_GPT_OUTPUT=True,
    RESET_CHAT=False,RETURN_RESPONSE=True)

[USER_MSG]


The current president of South Korea is directly elected for a five-year term with no possibility of re-election, and in case of a vacancy, a successor must be elected within sixty days. The current president of South Korea is the head of state and government, leading the State Council and serving as the commander-in-chief of the Armed Forces. The current president of South Korea is not mentioned in the given paragraph The presidential term in South Korea is currently set at five years since 1988, with the president being barred from re-election since 1981. The Provisional Government of the Republic of Korea established in September 1919 was recognized and succeeded by South Korea and its current Constitution. The paragraph describes the National Security Council and the Peaceful Unification Advisory Council in South Korea, their roles and membership. Yoon Suk Yeol, a former prosecutor general and member of the conservative People Power Party, became the president of South Korea on May 10, 2022, after winning the 2022 presidential election with a narrow 48.5% of the votes, defeating Lee Jae-myung from the Democratic Party. The paragraph discusses the controversial Advisory Council of Elder Statesmen in South Korea, which was expanded and elevated to cabinet rank before Roh Tae Woo became president, leading to suspicions of it being designed to benefit a specific individual. However, these suspicions became irrelevant when former President Chun withdrew from politics in November 1988. Using the information above, could you answer the following question? Who is the current president of South Korea?

[GPT_OUTPUT]


The current president of South Korea is Yoon Suk Yeol, who took office on May 10, 2022, after winning the 2022 presidential election.

In [ ]:
user_msg = "Could you explain about this little longer?"
response_content = GPT.chat(
    user_msg=user_msg,PRINT_USER_MSG=True,PRINT_GPT_OUTPUT=True,
    RESET_CHAT=False,RETURN_RESPONSE=True)